In [1]:
!pip install transformers
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install farasapy

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 14.6MB/s 
     |████████████████████████████████| 3.0MB 13.9MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=44a0bddb233906b1fed45e51bdb7aa7737141f24ae25aeb9464eba3d2faf0e6e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'arabert'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 198 (delta 112), reused 73 (delta 31), pack-reused 0
Receiving objects: 100% (198/198), 2.24 MiB | 2.19 MiB/s, done.
Resolving deltas: 100% (112/112), done.
     |████████████████████████████████| 112kB 2.8MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.8-cp36-none-any.whl size=110609 sha2

In [2]:
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess_arabert import never_split_tokens, preprocess
from farasa.segmenter import FarasaSegmenter
import torch

arabert_tokenizer = AutoTokenizer.from_pretrained(
    "aubmindlab/bert-base-arabert",
    do_lower_case=False,
    do_basic_tokenize=True,
    never_split=never_split_tokens)
arabert_model = AutoModel.from_pretrained("aubmindlab/bert-base-arabert") #you can replace the path here with the folder containing the the pytorch model

farasa_segmenter = FarasaSegmenter(interactive=True)


perform system check...
check java version...
Your java version is 11.0 which is compatiple with Farasa 
check toolkit binaries...
some binaries are not existed..
downloading zipped binaries...
 99%|█████████▉| 198M/200M [00:13<00:00, 14.8MiB/s]extracting...
toolkit binaries are downloaded and extracted.
Dependencies seem to be satisfied..
initializing [SEGMENT] task in INTERACTIVE mode...


/usr/local/lib/python3.6/dist-packages/farasa/__base.py:45: UserWarning: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.
  "Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases."


task [SEGMENT] is initialized interactively.


In [3]:
text= "الجو جميل اليوم"
text_preprocessed = preprocess( text,
                                do_farasa_tokenization = True,
                                farasa = farasa_segmenter,
                                use_farasapy = True) # if you want to use AraBERT v0.1 do_farasa_tokenization = False
print(text)
print("---------------------")
print(text_preprocessed)

الجو جميل اليوم
---------------------
ال+ جو جميل ال+ يوم


In [4]:
arabert_input = arabert_tokenizer.encode(text_preprocessed,add_special_tokens=True)
print(arabert_input)
print(arabert_tokenizer.convert_ids_to_tokens(arabert_input))
# you should ignore the fisrt and the last embeddings

[29756, 3000, 516, 15724, 3000, 7447, 29758]
['[CLS]', 'ال+', 'جو', 'جميل', 'ال+', 'يوم', '[SEP]']


In [5]:
tensor_input_ids = torch.tensor(arabert_input).unsqueeze(0)
print(tensor_input_ids)

tensor([[29756,  3000,   516, 15724,  3000,  7447, 29758]])


In [6]:
output = arabert_model(tensor_input_ids)

In [7]:
output[0].shape # batch_size x seq_len x emb_dim

torch.Size([1, 7, 768])

In [8]:
embeddings = output[0][0][1:-1]

In [9]:
print(embeddings.shape)
print(embeddings)

torch.Size([5, 768])
tensor([[ 0.3447,  1.3039,  1.3462,  ..., -0.4002,  0.5298,  0.2837],
        [-0.0045, -0.0522,  0.4295,  ..., -1.0770,  0.0021, -0.3939],
        [ 0.3940, -0.2821,  0.3980,  ..., -0.2659,  0.3846, -0.4675],
        [ 0.3452,  1.3047,  1.3463,  ..., -0.3996,  0.5308,  0.2845],
        [ 0.3442,  0.0026, -0.1066,  ..., -0.2442,  0.2427, -0.3018]],
       grad_fn=<SliceBackward>)
